# Welcome to Machine Learning!

This is the section of the class where we learn how to make a computer look at our data and identify aspects of the data that we didn't know to look for. The first section of this module begins with videos that give a brief background and introduction. In the following units, we'll start putting this vocabulary to use!

## Video 1: Introduction

You can find Casey's introduction to machine learning for GCB 535 here: https://youtu.be/Cj_giNsKZYc

## Video 2: Types of Machine Learning Methods

You can find Casey's discussion of different classes of machine learning methods here: https://youtu.be/4n2m3bLY2ps

### Prelab Questions:

_Q1: What type of question would you address with an unsupervised algorithm?_

_Q2: What type of algorithm would you use to find genes involved in mitochondrial biogenesis from a collection of microarray data if you have identified a few genes that play a role in the process?_

_Q3: Why?_

_Q4: For the situation described in Q2, what are the Features, Examples, Labels, and Predictions?_

## Video 3: Example of Supervised Machine Learning

You can find Casey's discussion how you might structure an analysis to use a supervised algorithm to predict the effective therapeutic dose of a drug here: https://youtu.be/9N19ogr9mZc

### Prelab Questions:

_Q1: Why are the samples in Video 2 features, while the samples here are examples?_

## Video 4: Example of Unupervised Machine Learning

You can find Casey's discussion how you might look for disease subtypes with unsupervised algorithms here: https://youtu.be/y400v_AAJSE

_Q1: What are the Features, Examples and Labels for the question discussed in Video 4?_